# Importing the libraries

In [ ]:
pip install pylibjpeg pylibjpeg-libjpeg pydicom

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pydicom as dicom
import os
from tqdm import tqdm
import cv2
from PIL import Image
from joblib import Parallel, delayed

# Reading the training dataset of breast cancer patients

In [3]:
train_data = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')


In [ ]:
print(train_data)

In [4]:
path_train_imgs = '/kaggle/input/rsna-breast-cancer-detection/train_images/'

patient_ids = train_data['patient_id']
image_ids = train_data['image_id']
sides  = train_data['laterality']

train_img_data = []
for pi, ii, leng in zip(patient_ids, image_ids, range(len(patient_ids))):
    train_img_data.append(path_train_imgs + str(pi) + '/' + str(ii) + '.dcm')
   
train_data['img_data'] = " "
train_img = []
def img_process(i,filename):
    
    ds = dicom.dcmread(filename)
    img = ds.pixel_array
    img_norm = cv2.normalize(img, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    img_resize = cv2.resize(img_norm,(128,128))
    img_fin = np.where(img_resize >= 0.9999, 0,img_resize)
    
   
    train_data.loc[i,'img_data'] = [img_fin]
    train_img.append(img_fin)



In [ ]:

Parallel(n_jobs=128)(
    delayed(img_process)(i,fname) for i,fname in zip(tqdm(range(len(train_img_data))),train_img_data)
)

 37%|███▋      | 20096/54706 [11:57<21:00, 27.47it/s]

In [ ]:
train_data.to_csv('/kaggle/working/training_img_data.csv')
